<h1>Project 6</h1>
Load all questions from a chapter<br/>
<h3>Requirements</h3>
Update project 5 and add support for taking a chapter name as input in the command line<br/>
Load all questions from the input chapter<br/>
Print all questions on the console<br/>
<h3>Error Handling</h3>
Take care of case where empty string is provided as input from command line<br/>
Take care of case where there are no questions corresponding to the provided chapter name<br/>


In [69]:
#Snippet to pick the chapter names

# Connect to MySQL Database
import mysql.connector
# Connect to MySQL Database
def connect_to_mysql_database():
    try:
        connection = mysql.connector.connect(
            host="localhost", 
            user="priya", 
            password="priya", 
            database="mydatabase"
        )
        if connection.is_connected():
            print("Connection to MySQL DB is successful!")
            return connection
    except Exception as e:
        print("Error while connecting to MySQL DB:", e)
        return None

def fetch_records(connection):
    mycursor = connection.cursor()
    mycursor.execute("select DISTINCT(chapter_name) from QUESTIONS;")
    temp_chapter_names_list = mycursor.fetchall()
    
    chapter_names_list = []
    
    for name in temp_chapter_names_list:
        chapter_names_list.append(name[0])
        
    print(f"chapter_names_list : {chapter_names_list}")
    return chapter_names_list


def fetch_questions_record(connection, name):
    mycursor = connection.cursor()
    name_value = []
    name_value.append(name)
    print(tuple(name_value))
    sql = "SELECT question_text FROM questions WHERE chapter_name = %s"

    mycursor.execute(sql, tuple(name_value))
    myresult = mycursor.fetchall()
    return myresult

connection = connect_to_mysql_database()
# fetch_records(connection)
# chapter_name = "WRONG"

# while chapter_name not in chapter_names_list:
#     chapter_name = input("Enter chapter name to pick questions from : ")
#     print(chapter_name)
#     if chapter_name in chapter_names_list:
#         break

# print(f"Chapter : {chapter_name}")
fetch_questions_record(connection, chapter_name)

Connection to MySQL DB is successful!
('Structure of Atom',)


[('Who proposed the plum pudding model of the atom?',),
 ('What is the charge of a neutron?',),
 ('Which experiment led to the discovery of the nucleus?',),
 ('What is the maximum number of electrons that can occupy a p -orbital?',),
 ('Who developed the quantum mechanical model of the atom?',),
 ('What is the principal quantum number primarily associated with?',),
 ("In Bohr's model, which property of electrons is quantized?",),
 ('Which particle was discovered first in the atomic model?',),
 ('What is the number of protons in an atom called?',),
 ('How many subshells are there in the 3rd energy level?',)]

In [87]:
import gdown
import re
import PyPDF2
import os
import mysql.connector

# Download file from Google Drive
def download_file(work_folder, actual_download_link_url):
    save_file_path = os.path.join(work_folder, "Chemistry Questions.pdf")
    
    try:
        gdown.download(actual_download_link_url, save_file_path, quiet=False)
        print(f"File downloaded successfully at path :{save_file_path}")
    except Exception as e:
        if not os.path.exists(work_folder):
            os.makedirs(work_folder)
        gdown.download(actual_download_link_url, save_file_path, quiet=True)
        print(f"File downloaded successfully at path :{save_file_path}")

# Connect to MySQL Database
def connect_to_mysql_database():
    try:
        connection = mysql.connector.connect(
            host="localhost", 
            user="priya", 
            password="priya", 
            database="mydatabase"
        )
        if connection.is_connected():
            print("Connection to MySQL DB is successful!")
            return connection
    except Exception as e:
        print("Error while connecting to MySQL DB:", e)
        return None

# Check if DB exists or not
def create_db_if_not_exists(connection):
    try:
        cursor = connection.cursor()
        cursor.execute("CREATE DATABASE IF NOT EXISTS mydatabase;")
    except Exception as e:
        print("Error while creating DB:", e)
        return None

# Create table if it doesn't exist
def create_table_if_not_exists(connection):
    cursor = connection.cursor()
    create_table_query = """
    CREATE TABLE IF NOT EXISTS QUESTIONS (
        id INT AUTO_INCREMENT PRIMARY KEY,
        subject_name VARCHAR(100),
        question_text TEXT,
        answer_options TEXT,
        correct_answer VARCHAR(10),
        chapter_name VARCHAR(100)
    );
    """
    cursor.execute(create_table_query)
    connection.commit()

def fetch_questions_from_db_based_on_user_input(connection, name):
    questions_list = []

    record_cursor = connection.cursor()
    name_value = []
    name_value.append(name)
    sql = "select question_text from QUESTIONS where chapter_name=%s;"
    record_cursor.execute(sql, tuple(name_value))
    questions_list = record_cursor.fetchall()
        
    # print(f"questions_list : {questions_list}")
    return questions_list
    
def fetch_chapters_name_list_from_db(connection):
    chapter_names_list = []

    record_cursor = connection.cursor()
    record_cursor.execute("select DISTINCT(chapter_name) from QUESTIONS;")
    temp_chapter_names_list = record_cursor.fetchall()
    
    for name in temp_chapter_names_list:
        chapter_names_list.append(name[0])
        
    print(f"\nchapter_names_list : {chapter_names_list}")
    return chapter_names_list
    
# Insert questions into the database
def insert_questions_into_db(connection, questions):
    cursor = connection.cursor()
    insert_query = """
    INSERT INTO QUESTIONS (subject_name, question_text, answer_options, correct_answer, chapter_name)
    VALUES (%s, %s, %s, %s, %s);
    """
    
    try:
        for question in questions:
            print(f"Inserting question: {question['question_text']}")
            cursor.execute(insert_query, (
                question['subject_name'],
                question['question_text'],
                question['answer_options'],
                question['correct_answer'],
                question['chapter_name']
            ))
        connection.commit()
        print(f"{len(questions)} questions inserted successfully.")
    except Exception as e:
        print("Error inserting data into database:", e)

def main():
    # Connect to MySQL Database
    connection = connect_to_mysql_database()
    
    if connection:
        create_db_if_not_exists(connection)
        # Create table if it doesn't exist
        create_table_if_not_exists(connection)

    # Extract questions from the PDF
    # 1.Store a PDF file in a folder called “/content” 
    download_link_url = "https://drive.google.com/file/d/1OVFr62PNO5-PcnWwFO1Frp0K_0UX6ENc/view?usp=drive_link"
    google_download_file_id = download_link_url.split("/")[5]
    # Corrected URL format for downloading the file
    actual_download_link_url = f'https://drive.google.com/uc?id={google_download_file_id}'
    print("actual_download_link_url : ",actual_download_link_url)
    work_folder = "./content"
    
    # Uncomment to download the file
    download_file(work_folder, actual_download_link_url)

    pdf_path = os.path.join(work_folder, "Chemistry Questions.pdf")
    # questions = extract_questions_from_pdf(pdf_path)

    # if questions:
    #     # Insert extracted questions into the database
    #     insert_questions_into_db(connection, questions)
    chapters_name_list = fetch_chapters_name_list_from_db(connection)
    chapter_name = "WRONG" #initial wrong value

    while chapter_name not in chapter_names_list:
        chapter_name = input("Enter chapter name to pick questions from : ")
        if chapter_name in chapter_names_list:
            break

    questions_list = fetch_questions_from_db_based_on_user_input(connection, chapter_name)
    print("\nQuestions list is as follows : \n")
    for questions in questions_list:
        print(questions)

    # Close the database connection
    connection.close()

if __name__ == "__main__":
    main()


Connection to MySQL DB is successful!
actual_download_link_url :  https://drive.google.com/uc?id=1OVFr62PNO5-PcnWwFO1Frp0K_0UX6ENc


Downloading...
From: https://drive.google.com/uc?id=1OVFr62PNO5-PcnWwFO1Frp0K_0UX6ENc
To: /Users/priprabh/Downloads/Priyanka/Learnings & Training/Python/Udemy Python/Python-Coding-Exercises/Python-Coding-Exercises/Assignment/Project6/content/Chemistry Questions.pdf
100%|████████████████████████████████████████| 279k/279k [00:00<00:00, 2.34MB/s]


File downloaded successfully at path :./content/Chemistry Questions.pdf

chapter_names_list : ['Basic concepts of chemistry', 'Structure of Atom', 'Classification of elements and periodicity', 'States of Matter']


Enter chapter name to pick questions from :  Classification of elements and periodicity



Questions list is as follows : 

('Who is credited with creating the modern periodic table?',)
('What is the basis of classification in the modern periodic table?',)
('Which group of the periodic table contains the noble gases?',)
('Elements in the same group of the periodic table generally have similar...',)
('What term describes the horizontal rows of the periodic table?',)
('The alkali metals are found in which group of the periodic table?',)
('What characteristic is common to all elements in Group 17 (halogens)?',)
('Which of the following elements is a transition metal?',)
('What is the general trend for atomic radius as you move down a group in the periodic table?',)
('Which of the following elements is located in Period 3 and Group 16?',)
('Which of the following bonds is the strongest?',)
('What is the geometry of a molecule with sp² hybridization?',)
('The bond angle in methane (CH ₄) is:',)
('Which of the following molecules exhibits resonance?',)
('Which molecule has the hi